In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu 

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-03 04:09:03--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.53MB/s    in 0.2s    

2023-03-03 04:09:04 (4.53 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
# url = https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
from pyspark import SparkFiles
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), header=True, inferSchema=True, sep='\t')

# Show DataFrame
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [5]:
# Get the number of rows in the DataFrame.
print(df.count())

1741100


# Transform the Data

## Create the "review_id_table".

In [11]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
tool_review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
tool_review_id_df.show(20)
print("Number of Rows: " + str(tool_review_id_df.count()))

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2UM5QMHBHC90Q|   15785389|B00H5U9ZD6|     115362950| 2015-08-31|
|  RF0D1LEIF6L7|   47910848|B001TJGCS0|     570955425| 2015-08-31|
|  RM6YKIWQVNSY|   36328996|B000NIK8JW|     128843593| 2015-08-31|
|R1RL3L68ASPS36|   51785809|B008ZYRGUA|     407828107| 2015-08-31|
|R1U4XFBFAG34CY|   40757491|B00K5CA0GC|     490746675| 2015-08-31|
|R3KFIK8P0I91PL|   35544833|B00AIJAA94|     148352067| 2015-08-31|
| RENOAY76PPK1O|   16474909|B00JKEI6K8|     331801084| 2015-08-31|
| RINV884I0NL5V|   22601598|B00AGCHVS6|     471514859| 2015-08-31|
| R5KJH6CXZH2PX|   16129808|B002S0O7U4|     162253576| 2015-08-31|
| RO69JF6QWD0W1|   24382335|B0084YHXMW|      69530650| 2015-08-31|
|R3L9NQBH3FI55C|   49796324|B00MLSS1SW|     916693555| 2015-08-31|
| R4YH95YPHVU0C|   33289687|B00D4WLS2A|      39333316| 2015-08

## Create the "products" Table

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
tool_products_df = df.select(["product_id", "product_title"]).dropDuplicates()
tool_products_df.show(20)
print("Number of Rows: " + str(tool_products_df.count()))

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000JNNWQ2|IRWIN Tools VISE-...|
|B000OECQOK|Titan 17005 Stain...|
|B0045W0MD4|The American Red ...|
|B00DQQ9ET8|de.power DP-010AA...|
|B001P30BQE|DEWALT DG5204 Pro...|
|B0002TKID8|Channellock GLS-1...|
|B003O684GO|Tajima AC-500B 3/...|
|B00T96T070|UltraFire WF-68B ...|
|B001445114|Stiletto AG-102 A...|
|B00002X1ZX|Stanley Tools 28-...|
|B0006IE7OQ|Neiko 00561A Heav...|
|B00C8STBPC|ZFE 42Pcs Brass S...|
|B00004SUQN|Malco M2006 Max20...|
|B000TCZUWM|Et18 RS Mizar Ele...|
|B000189RGO|Eklind 13218 Metr...|
|B009SB5RHS|Huntsman® - W20 4...|
|B0002IXQHO|Robert Sorby 67HS...|
|B004Q05TTC|Superior Tool 375...|
|B00012Y38W|Bondhus 20199 Bal...|
|B003YL4954|PORTER-CABLE NS18...|
+----------+--------------------+
only showing top 20 rows

Number of Rows: 183811


## Create the "customers" Table

In [8]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
tool_customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
tool_customers_df.show(20)
print("Number of Rows: " + str(tool_customers_df.count()))

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45978717|             2|
|   43622307|             1|
|     740678|             4|
|   29045703|             1|
|   52484548|             3|
|   42560427|             4|
|   17067926|             1|
|   10093406|             1|
|   44979559|             1|
|   19432125|             1|
|   26079415|             1|
|   29931671|             1|
|   12945150|             3|
|   12036434|             1|
|   14230926|             1|
|   45015535|             1|
|   39064792|             1|
|   20709090|             1|
|   45074906|             1|
|   26767269|             2|
+-----------+--------------+
only showing top 20 rows

Number of Rows: 1053515


## Create the "vine_table".

In [9]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
tool_vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
tool_vine_df.show(20)
print("Number of Rows: " + str(tool_vine_df.count()))

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|
|  RF0D1LEIF6L7|          4|            0|          0|   N|
|  RM6YKIWQVNSY|          1|            6|          6|   N|
|R1RL3L68ASPS36|          4|            0|          0|   N|
|R1U4XFBFAG34CY|          5|            0|          0|   N|
|R3KFIK8P0I91PL|          5|            0|          0|   N|
| RENOAY76PPK1O|          5|            0|          0|   N|
| RINV884I0NL5V|          1|            0|          0|   N|
| R5KJH6CXZH2PX|          5|            0|          0|   N|
| RO69JF6QWD0W1|          5|            0|          0|   N|
|R3L9NQBH3FI55C|          4|            1|          1|   N|
| R4YH95YPHVU0C|          5|            0|          0|   N|
|R10M1WWDDQBGG2|          2|            0|          0|   N|
| RV3KWQBTNIO62|          5|            

# Load

In [10]:
mode = "append"
jdbc_url="jdbc:postgresql://database-1.ci0x99fhs29y.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", "password": "Andromon", "driver":"org.postgresql.Driver"}

In [12]:
# Write review_id_df to table in RDS
tool_review_id_df.write.jdbc(url=jdbc_url, table='tool_review_id', mode=mode, properties=config)

In [13]:
# Write products_df to table in RDS
tool_products_df.write.jdbc(url=jdbc_url, table='tool_products', mode=mode, properties=config)

In [14]:
# Write customers_df to table in RDS
tool_customers_df.write.jdbc(url=jdbc_url, table='tool_customers', mode=mode, properties=config)

In [15]:
# Write vine_df to table in RDS
tool_vine_df.write.jdbc(url=jdbc_url, table='tool_vine', mode=mode, properties=config)